In [1]:
import numpy as np
import matplotlib.pyplot as plt

from NanTex_backend.Util.pyDialogue import pyDialogue as pD
from NanTex_backend.data_preparation.overlay_generation import OverlayGenerator

# LINUX

In [ ]:
## Glob data
from glob import glob

root_feature_1: str = "path/to/feature_1"
root_feature_2: str = "path/to/feature_2"
root_feature_3: str = "path/to/feature_3"

root_features: list[str] = [root_feature_1, root_feature_2, root_feature_3]
root_features = [glob(f"{root}/*.npy") for root in root_features]

In [ ]:
## setup
OLGen: OverlayGenerator
OLGen = OverlayGenerator.from_glob(
    *root_features, data_path_out=None, mode="rotation", multi_core=True, DEBUG=True
)

# Windows

In [2]:
## setup
OLGen: OverlayGenerator
OLGen = OverlayGenerator.from_explorer(
    data_path_out=None, mode="overlay", multi_core=False, DEBUG=True, dtype_out=np.uint16
)

Please select files for feature #1...
Please select files for feature #2...
Please select files for feature #3...
No more features to select...
Full image mode detected. Setting output dtype to uint16.
User input detected. Overriding output dtype.
Full image mode detected. Auto-standardization disabled.
Auto-standardization disabled.
Loading data...


Loading data...:   0%|          | 0/2 [00:00<?, ?it/s]

Loading npy data from Z:/NanTex/wuerzburg_regeneration/CLA_test/CLA_1_cut.npy...
Loading npy data from Z:/NanTex/wuerzburg_regeneration/CLA_test/CLA_1_cut.npy...
Setting up metadata...
Checking outpath...
Retrieving outpath...


In [3]:
OLGen.metadata

{'x_lim': 6541,
 'y_lim': 4122,
 'dtype_out': numpy.uint16,
 'mode': 'overlay',
 'multi_core': False,
 'patches': None,
 'img_size': (4096, 4096),
 'patchsize': (64, 64),
 'augment': False,
 'DEBUG': True,
 'sleeptime': 0.0,
 'patch_content_ratio': 0.0,
 'disable_auto_standardization': True}

In [3]:
OLGen.patches = 2
OLGen.dtype_out = np.float32
OLGen.disable_auto_standardization = False

In [6]:
OLGen.__pad_input_imgs__()

Padding input images...


In [7]:
OLGen.data_in['feature_1']

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], shape=(4096, 4096), dtype=uint8)]

In [4]:
OLGen.generate_punchcards()

Generating punchcards...
Initializing patch mode...
Augmentations disabled. Falling back to patch generation only.
Setting up patch pipeline without augmentation...


In [4]:
OLGen.generate_overlay()

Performing startup checks...
Starting up checks...
Auto-standardization enabled.
Padding input images...
Normalizing input images...
Generating punchcards...
Initializing patch mode...
Augmentations disabled. Falling back to patch generation only.
Setting up patch pipeline without augmentation...
Generating overlay...


Generating Single Core Patch Overlays...:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
from NanTex_backend.data_preparation.overlay_generation import OVERLAY_HELPER

In [25]:
def __generate_patches__(punchcard, data_in, overlay_worker):
    # generate base overlay (e.g. __generate_stack__)
    tmp = overlay_worker(punchcard=punchcard, data_in=data_in)
    img = tmp[-1,:,:] # <- overlay is always the last image in the stack
    masks = list(tmp[:-1,:,:]) # <- all other images are masks

    # generate patches
    patch_collector= {i:None for i in range(punchcard["patches"])}
    while any(v is None for v in patch_collector.values()):

        # extract patch
        augmented = punchcard["augmentation"](image=img, masks=masks)
        augmented_img = augmented["image"]
        augmented_masks = augmented["masks"]
        
        # check for content
        if not OVERLAY_HELPER.__check_img_content__(augmented_img, punchcard["patch_content_ratio"]):
            continue

        # find empty patch
        for i, patch in patch_collector.items():
            if patch is None:
                patch_collector[i] = np.stack([augmented_masks, augmented_img], axis=0)
                break
            
    return patch_collector

In [13]:
punchcard = OLGen.data_punchcards['2_feature_overlay_(0, 0)']
data_in = OLGen.data_in

In [17]:
overlay_worker = OVERLAY_HELPER.__generate_stack__
tmp = overlay_worker(punchcard=punchcard, data_in=data_in)
img = tmp[-1,:,:] # <- overlay is always the last image in the stack
masks = list(tmp[:-1,:,:]) # <- all other images are masks


In [22]:
augmented = punchcard["augmentation"](image=img, masks=masks)
augmented_img = augmented["image"]
augmented_masks = augmented["masks"]

In [29]:
augmented_masks

[array([[  0.,   0.,   0., ...,   0.,   0., 104.],
        [  0.,   0.,   0., ...,   0.,   0.,  19.],
        [  0.,   0.,   0., ...,  21.,   0.,  12.],
        ...,
        [  0.,   9.,  30., ...,  14.,  36.,   0.],
        [  0.,   0.,   0., ...,  28.,   0.,  30.],
        [  0.,   0.,   0., ...,  21.,   0.,   6.]],
       shape=(64, 64), dtype=float32),
 array([[  0.,   0.,   0., ...,   0.,   0., 104.],
        [  0.,   0.,   0., ...,   0.,   0.,  19.],
        [  0.,   0.,   0., ...,  21.,   0.,  12.],
        ...,
        [  0.,   9.,  30., ...,  14.,  36.,   0.],
        [  0.,   0.,   0., ...,  28.,   0.,  30.],
        [  0.,   0.,   0., ...,  21.,   0.,   6.]],
       shape=(64, 64), dtype=float32)]

In [31]:
np.stack([*augmented_masks, augmented_img], axis=0).shape

(3, 64, 64)

In [23]:
augmented_img

array([[-0.2809591 , -0.2809591 , -0.2809591 , ..., -0.2809591 ,
        -0.2809591 ,  3.478944  ],
       [-0.2809591 , -0.2809591 , -0.2809591 , ..., -0.2809591 ,
        -0.2809591 ,  0.40594625],
       [-0.2809591 , -0.2809591 , -0.2809591 , ...,  0.47825208,
        -0.2809591 ,  0.15287586],
       ...,
       [-0.2809591 ,  0.04441711,  0.8036283 , ...,  0.22518168,
         1.0205458 , -0.2809591 ],
       [-0.2809591 , -0.2809591 , -0.2809591 , ...,  0.73132247,
        -0.2809591 ,  0.8036283 ],
       [-0.2809591 , -0.2809591 , -0.2809591 , ...,  0.47825208,
        -0.2809591 , -0.06404163]], shape=(64, 64), dtype=float32)

In [ ]:
OLGen.generate_overlay()

Performing startup checks...
Starting up checks...
Auto-standardization enabled.
Padding input images...
Normalizing input images...
Generating punchcards...
Initializing patch mode...
Augmentations disabled. Falling back to patch generation only.
Setting up patch pipeline without augmentation...
Generating overlay...


Generating Single Core Patch Overlays...:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: all input arrays must have the same shape

In [ ]:
OLGen.multi_core = False
OLGen.mode = "rotation"
OLGen.generate_overlay(patchsize=(2048, 2048))

In [4]:
OLGen.mode = "rotation"
OLGen.multi_core = True
OLGen.metadata["sleeptime"] = 0.0
OLGen.metadata["dtype"] = np.uint16
OLGen.setup_multi_core(num_cpu=12, num_gpu=0, launch_dashboard=True)

Setting up Ray...


2025-08-21 16:44:53,808	INFO worker.py:1918 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


Ray setup complete...
Ray Dashboard: 127.0.0.1:8266


In [ ]:
OLGen.generate_overlay(patchsize=(512, 512), max_val=255)

Padding input images...
Normalizing input images...
Generating punchcards...
Generating overlay...
Checking Ray Status...
Offloading data_in to Ray...
Offloading punchcards to Ray...
Offloading data_outpath to Ray...


Scheduling Workers:   0%|          | 0/12288 [00:00<?, ?it/s]

Setting up progress monitors...


Workers:   0%|          | 0/12288 [00:00<?, ?it/s]

CPU usage:   0%|          |

RAM usage:   0%|          |

Listening to Ray Progress...


In [ ]:
OLGen.shutdown_multi_core()

In [ ]:
## checking the setup
print(OLGen.metadata)
print(OLGen.data_path_out)
print(OLGen.data_paths_in)

plt.imshow(OLGen.data_in["feature_3"][0])

## DEBUG

In [17]:
import albumentations as A

t = [A.HorizontalFlip(p=1), A.VerticalFlip(p=1), A.RandomRotate90(p=1)]
s = A.RandomCrop(height=256, width=256)
t = A.Compose(t)

In [3]:
OLGen.augment = True
OLGen.patches = 4
OLGen.patchsize = (256, 256)
#OLGen.augmentation_pipeline = t

In [7]:
OLGen.augmentation_pipeline

Compose([
  RandomCrop(p=1.0, border_mode=0, fill=0.0, fill_mask=0.0, height=256, pad_if_needed=False, pad_position='center', width=256),
  HorizontalFlip(p=0.5),
  VerticalFlip(p=0.5),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={}, is_check_shapes=True)

In [6]:
## Punchcard generation
OLGen.generate_punchcards()
OLGen.data_punchcards

Generating punchcards...
Initializing patch mode...
Initializing augmentation pipeline...
Setting up patch pipeline with augmentation...


{'2_feature_rotation_(0, 0, 0, 0)': {'feature_1': 0,
  'feature_2': 0,
  'rotation': (0, 0),
  'dtype': numpy.uint16,
  'augmentation': Compose([
    RandomCrop(p=1.0, border_mode=0, fill=0.0, fill_mask=0.0, height=256, pad_if_needed=False, pad_position='center', width=256),
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
  ], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={}, is_check_shapes=True),
  'patches': 4},
 '2_feature_rotation_(0, 0, 0, 1)': {'feature_1': 0,
  'feature_2': 0,
  'rotation': (0, 1),
  'dtype': numpy.uint16,
  'augmentation': Compose([
    RandomCrop(p=1.0, border_mode=0, fill=0.0, fill_mask=0.0, height=256, pad_if_needed=False, pad_position='center', width=256),
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
  ], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={}, is_check_shapes=True),
  'patches': 4},
 '2_feature_rotation_(0, 0, 0, 2)': {'feature_1': 0,
  'feature_2': 0,
  'rotation': (0, 2),
  'dtype': numpy.uint

In [ ]:
OLGen.data_paths_in

In [ ]:
## Punchcard generation
OLGen.mode = "rotation"
OLGen.multi_core = False
OLGen.generate_punchcards()
OLGen.data_punchcards

In [ ]:
from NanTex_backend.data_preparation.overlay_generation import OVERLAY_HELPER

In [ ]:
punchcard = OLGen.data_punchcards["3_feature_rotation_(0, 0, 0, 3, 0, 2)"]
data_in = OLGen.data_in

out = [
    data_in[key][value]
    for key, value in punchcard.items()
    if key not in ["rotation", "dtype"]
]
print(out[0].shape, out[1].shape, out[2].shape)
out = [np.rot90(img, k=punchcard["rotation"][i]) for i, img in enumerate(out)]
print(out[0].shape, out[1].shape, out[2].shape)

In [ ]:
OVERLAY_HELPER.__generate_stack_rotation__(
    data_in=OLGen.data_in,
    punchcard=OLGen.data_punchcards["3_feature_rotation_(0, 0, 0, 3, 0, 2)"],
)

In [ ]:
OLGen.generate_overlay(patchsize=(512, 512))

In [ ]:
OLGen.data_in["feature_1"][0].dtype

In [ ]:
OLGen.__pad_input_imgs__(patchsize=(256, 256))
OLGen.data_in["feature_1"][0].dtype

In [ ]:
OLGen.__normalize_input_imgs__(max_val=255)
OLGen.data_in["feature_1"][0].dtype

In [ ]:
## Generate output
OLGen.mode = "overlay"
OLGen.generate_punchcards()
OLGen.generate_overlay(patchsize=(256, 256))

In [ ]:
## Data Inspector
files: list = pD.askFILES()
files = [np.load(f) for f in files]

fig, axs = plt.subplots(len(files), files[0].shape[0], figsize=(20, 10))
for j, file in enumerate(files):
    for i in range(files[0].shape[0]):
        axs[j, i].imshow(file[i])
        axs[j, i].axis("off")
        axs[j, i].set_title(f"File {j} - Patch {i} - Max: {np.max(file[i]):.2f}")